## Facility Location

with integer programming and Gurobi

Source: http://examples.gurobi.com/kidney-exchange/

简单的生产规划问题，采用 piecewise-linear 目标函数。

in this example we'll solve a simple production scheduling problem and demonstrate the use of piecewise-linear objectives in Gurobi.


<img height=400 width=600 src=http://examples.gurobi.com/kidney-exchange/screenshot.png>


## Problem Description

piecewise-linear 函数用于进行函数。可选取 n 个点来模拟目标函数，n 越大模拟越精确。

Before presenting the example, we briefly review piecewise-linear functions. Piecewise-linear functions can be used to approximate arbitrary (nonlinear) functions. As an example, the function
 
$$
f(x)=−(x−2.5)^2+sin(kx)+3 
$$
 
 
and its piecewise-linear approximation are shown on the visualization below. You can vary the number of sample points $n$ and the parameter $k$ to see how the piecewise-linear approximation changes.

如果函数非凸，此问题可以用 piecewise-linear 目标函数作为MIP求解； 如果是凸问题，则直接可以用LP求解。

The function $f(x)$ is not convex. But $f(x)$ can still be minimized/maximized with Gurobi by invoking piecewise-linear objectives. The problem will be transformed to a MIP and solved. If $f(x)$ is convex, the model is directly solved as an LP.

Piecewise-linear objectives arise naturally in many different applications. In this example, we will see how piecewise-linear objectives are used to solve problems that include soft constraints.

We consider a pulp and paper factory which uses wood as a raw material to produce different types of paper, cardboard and pulp. Each product has a cost and can be produced at a certain rate. The goal is then to decide, given the demand, the amount of each item to produce to maximize the profit.

The factory can only run for a limited number of hours. To model this, we could add a hard constraint. For example, limiting the number of work hours in a week to 50. In reality, it is often the case that we can go beyond the time limit, but only if we pay overtime costs. So if 55 hours of work were done in a week, the first 50 would incur no extra cost, but we would pay a penalty of 100 $/hour for the final 5 hours. This is a soft constraint, because it can be violated if a penalty is payed.

Piecewise-linear objectives also arise in other contexts, for example in modelling reversible activites, piecewise-linear costs, approximating nonlinear functions, and many more.

## Implementation

In [1]:
from gurobipy import *

def cost(x, limithours, penalty):
    if x < limithours:
        return basecost
    else:
        return basecost + (x-limithours)*penalty

# Example data
rate = [50,40]; revenue = [25,32]; limit = [1200,920];
limithours = 20; maxhours = 40; penalty = 100; basecost = 500;

n = len(rate) # number of products

m = Model()

# Add variables
x = {}

for i in range(n):
    x[i] = m.addVar(ub = limit[i], vtype=GRB.CONTINUOUS, name="x%d" % i)

t = m.addVar(vtype=GRB.CONTINUOUS, name="t")

m.update()

# Add constraints
m.addConstr(t == quicksum( x[j]/rate[j] for j in range(n)))

m.addConstr(t <= maxhours)

# 使用 传统 setObjective
# Set objective
# 由于使用了下面setPWLObj，所以这句应该是无效的
m.setObjective( quicksum(revenue[i]*x[i] for i in range(n)), GRB.MAXIMIZE)

# 附加 setPWLObj 
# Set piecewise linear objective
nPts = 101
ti = []
costi = []
lb = 0
ub = maxhours;

for i in range(nPts):
    ti.append(lb + (ub - lb) * i / (nPts - 1))
    costi.append(-cost(ti[i], limithours, penalty))

m.setPWLObj(t, ti, costi)

m.optimize()

Optimize a model with 2 rows, 3 columns and 4 nonzeros
Model has 1 piecewise-linear objective term
Variable types: 3 continuous, 0 integer (0 binary)
Coefficient statistics:
  Matrix range     [2e-02, 1e+00]
  Objective range  [2e+01, 3e+01]
  Bounds range     [9e+02, 1e+03]
  RHS range        [4e+01, 4e+01]
Found heuristic solution: objective -500
Presolve removed 1 rows and 0 columns
Presolve time: 0.00s
Presolved: 2 rows, 43 columns, 44 nonzeros
Variable types: 43 continuous, 0 integer (0 binary)

Root relaxation: objective 4.819000e+04, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    48190.000000 48190.0000  0.00%     -    0s

Explored 0 nodes (1 simplex iterations) in 0.02 seconds
Thread count was 4 (of 4 available processors)

Solution count 2: 48190 -500 
Pool objective bound 48190

Optimal solution found (tolerance 1.

## Optimal Result

TOOD：objVal变化了。但是为什么Vars没有变化，还都是850, 920？ 

``` 
使用 传统 setObjective
最优值为: 50690.0
x0 850.000000
x1 920.000000
t 40.000000

附加 setPWLObj
最优值为: 48190.0
x0 850.000000
x1 920.000000
t 40.000000
```

In [2]:
print "最优值为:", m.objVal
for v in m.getVars():
    if v.X != 0:
        print("%s %f" % (v.Varname, v.X))

最优值为: 48190.0
x0 850.000000
x1 920.000000
t 40.000000


## Live Demo

Below is a visualization of our problem.

每个圆圈代表一个产品，下方是生产率、预算和生产限制。

Each product is represented by a circle, with the given production rate, revenue and production limit written below.

piecewise阈值(piecewise-linear penalty )函数是可以修改的。

You can modify the piecewise-linear penalty function by dragging the points on the graph.

Click "Compute" to find the optimal production schedule. If a product is built, the circle representing it will be filled.
